In [ ]:
import pandas as pd
import torch
import numpy as np
from qdrant_client import QdrantClient
from transformers import pipeline
from typing import Optional, Union, List, Tuple
import torch.nn.functional as F
import logging
from qdrant_client import QdrantClient, models
from qdrant_client.http.models import VectorParams, Distance, PointStruct, Filter, FieldCondition, MatchValue
from huggingface_hub import login
from HF_t import hf_token_read
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification,T5Tokenizer,T5ForConditionalGeneration,AutoModelForSeq2SeqLM
import sentencepiece as spm
from auto_gptq import AutoGPTQForCausalLM, BaseQuantizeConfig
import re


# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [2]:
# Set device if not provided
device = None
if device is None:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Connect to the Vector Database

In [14]:
# Initilizae the Qdrant
from qdrant_client import QdrantClient, models
from qdrant_client.http.models import VectorParams, Distance, PointStruct, Filter, FieldCondition, MatchValue

#initialize with a local path to persist data on disk without a server:
#client = QdrantClient('./PetHealth_Chatbot/Vector_Database')
# Connect to a local Qdrant instance
client = QdrantClient(url="http://localhost:6333") # Or specify your actual local host/port

collection_name = "vet_notes"

collection_info = client.get_collection(collection_name=collection_name)

INFO:httpx:HTTP Request: GET http://localhost:6333 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://localhost:6333/collections/vet_notes "HTTP/1.1 200 OK"


# Loading Models
First, we'll load two different models:
1. VetBERT model for generating embeddings (vector representations)
2. Classification model for predicting conditions
3. Generating tex model for Summarization

In [6]:
# 1. Load VetBERT model for embeddings
print("Loading VetBERT model for embeddings...")
vetbert_model = AutoModel.from_pretrained("havocy28/VetBERT")
vetbert_tokenizer = AutoTokenizer.from_pretrained("havocy28/VetBERT")

# Check if a GPU is available and move the model to the GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# vetbert_model.to(device)
print("VetBERT model loaded successfully")

Loading VetBERT model for embeddings...
VetBERT model loaded successfully


In [9]:
#2. loading the model from hugging face that trained on owner notes for classification
#Use the token for reading to load the model in huging face
from HF_t import hf_token_read

# In your notebook
# from huggingface_hub import login
# from HF_t import hf_token_read  # Import your token securely

login(token=hf_token_read)

# Replace "your-username/my-awesome-fine-tuned-model" with your actual repository ID
repo_id = "fdastak/model_calssification" 

# Load the fine-tuned model and its tokenizer
model_Class= AutoModelForSequenceClassification.from_pretrained(repo_id)
tokenizer_class = AutoTokenizer.from_pretrained(repo_id)

In [53]:
#3. Loading the model for summarization...
print("Loading ClinicalT5-base model for summarization...")
# Load tokenizer and model from Hugging Face hub
# Load small T5 model (t5-small or t5-base)
model_name = "t5-small"

tokenizer_summ = T5Tokenizer.from_pretrained(model_name)
model_summ = T5ForConditionalGeneration.from_pretrained(model_name)



Loading ClinicalT5-base model for summarization...


In [ ]:
#4. Loading the model for explaination...
print("Loading flan-t5-base model for expaination...")
# Load tokenizer and model from Hugging Face hub
# Load the model (flan-t5-base)
model_name = "google/flan-t5-base"  # or flan-t5-large for better performance
tokenizer_textg = AutoTokenizer.from_pretrained(model_name)
model_textg = AutoModelForSeq2SeqLM.from_pretrained(model_name)

tokenizer_textg = T5Tokenizer.from_pretrained(model_name)
model_textg = T5ForConditionalGeneration.from_pretrained(model_name)

Loading ClinicalT5-base model for expaination...


# Create an embeding class that can be used by differnt AI agent

In [15]:
# Define the mixin with the shared embedding method
class VetBERTMixin:
    def get_vetbert_embeddings(
        self, 
        user_input: str, 
        return_numpy: bool = True
    ) -> Union[torch.Tensor, np.ndarray]:
        """
        Generate embeddings using VetBERT model.

        Args:
            user_input (str): Text to embed
            return_numpy (bool): Return numpy array if True, else torch tensor
        
        Returns:
            torch.Tensor or np.ndarray: Embedding vector
        """
        self.model.eval()
        inputs = self.tokenizer(
            user_input,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=512
        )
        inputs = {k: v.to(self.device) for k, v in inputs.items()}

        with torch.no_grad():
            outputs = self.model(**inputs)
            embeddings = outputs.last_hidden_state[:, 0, :]  # CLS token

        if return_numpy:
            embeddings = embeddings.cpu().numpy()
        return embeddings

# Agent System

We'll create two agents that work together:
1. **Classification Agent**: Identifies the pet's condition from user input
2. **Retrieval Agent**: Finds relevant veterinary notes based on the identified condition

In [193]:
class ClassificationAgent:
    def __init__(self, model, tokenizer, device):
        self.device = device
        self.model = model.to(self.device) # Move model to device
        self.tokenizer = tokenizer
        self.label_to_condition = {
            0: "digestive issues",
            1: "ear infections",
            2: "mobility problems",
            3: "parasites",
            4: "skin irritations"
        }

    def predict_condition(self,user_input) -> tuple:
        """
        Predict the condition for a given input text using the classification model.
            
        Returns:
            tuple: (predicted_label, confidence_score)
        """
        try:
            # Set to eval mode
            self.model.eval()
            
            # Tokenize input for classifier
            inputs = self.tokenizer(
                user_input,
                return_tensors="pt",
                padding=True,
                truncation=True,
                max_length=512
            )
            inputs = {k: v.to(self.device) for k, v in inputs.items()}
            
            # Get predictions
            with torch.no_grad():
                outputs = self.model(**inputs)
                logits = outputs.logits
                probs = F.softmax(logits, dim=-1)
                confidence_score, pred_label = torch.max(probs, dim=1)
                
                logger.info(f"Successfully generated prediction with confidence {confidence_score.item():.4f}")
                return pred_label.item(), confidence_score.item()
                
        except Exception as e:
            logger.error(f"Prediction failed: {str(e)}")
            raise

    def identify_condition(self, user_input: str) -> tuple:
        """
        Identify pet's condition from user input.
            
        Args:
        user_input (str): Description of pet's symptoms
                
        Returns:
        tuple: (condition_name, confidence_score)
            """
        try:
            # Get prediction
            predicted_label, confidence = self.predict_condition(user_input)
                
            # Convert to condition name
            condition_name = self.label_to_condition.get(predicted_label, "unknown condition")
                
            logger.info(f"Identified condition: {condition_name} with confidence: {confidence:.4f}")
            return condition_name, confidence
                
        except Exception as e:
            logger.exception(f"Error in classification: {str(e)}")
            raise

class RetrievalAgent(VetBERTMixin):
    def __init__(self, model, tokenizer, device, qdrant_client, collection_name):
        self.device = device
        self.model = model.to(self.device) # Move model to device
        self.tokenizer = tokenizer
        self.client = qdrant_client
        self.collection_name = collection_name

    def find_similar_cases(self, user_input: str, condition: str, limit: int = 3) -> list:
        """
        Find similar veterinary cases based on input and condition.
        
        Args:
            user_input (str): User's description of symptoms
            condition (str): Identified condition to filter by
            limit (int): Number of similar cases to retrieve
            
        Returns:
            list: Similar cases with their scores
        """

        try:
            # Generate embeddings for the query
            query_vector = self.get_vetbert_embeddings(
                user_input,
                return_numpy=True
            )
            
            # Set up condition filter
            condition_filter = Filter(
                must=[FieldCondition(key="condition", match=MatchValue(value=condition))]
            )
            
            # Search for similar cases
            results = self.client.search(
                collection_name=self.collection_name,
                query_vector=query_vector[0],
                limit=limit,
                query_filter=condition_filter,
                with_payload=True
            )
            # qp = client.query_points(
            # collection_name=collection_name,
            # query=query_vector[0].tolist(),
            # limit=3,
            # query_filter=filter_by_category,
            # with_payload=True,
            #  )
            # results = qp.points

            
            logger.info(f"Found {len(results)} similar cases for condition: {condition}")
            return results
            
        except Exception as e:
            logger.exception(f"Error in retrieval: {str(e)}")
            raise
    
class communicationAgent():
    def __init__(self,model,tokenizer,device):
        self.device = device
        self.model = model.to(self.device) # Move model to device
        self.tokenizer = tokenizer

    def output_vet_assistant(self,case):
        # Prepare the input text for T5
        finding_s=''
        for i in range(len(case)):
            
        # 4. Create an effective prompt for the task
        # The prompt should be clear and instructive, asking the model to perform two steps:
        # a) Identify the clinical terms.
        # b) Explain each term.
            prompt_template = """
                Identify and explain clinical entities in the text provided.And provide a short, clear explanation for each term.
                example :Gastroenteritis: inflammation of the stomach and intestines causing vomiting and diarrhea.
                Text: {clinical_text}"""


            prompt = prompt_template.format(clinical_text=case[i].payload['text'])
            inputs= self.tokenizer(prompt, return_tensors="pt", max_length=512, truncation=True)
            inputs= {k: v.to(device) for k, v in inputs.items()}
            # Generate the output
            outputs = self.model.generate(**inputs, max_new_tokens=50,  do_sample=True, 
                temperature=0.5,
                top_p=0.9)
            # Decode and print the output
            decoded_output = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
            finding_s=f"{len(case)-i}.{case[i].payload['text']}\n"+f"{decoded_output}\n"+finding_s
        
      

        # Carefully craft a conversational question
        appointment_question = (
            "\n\nWould you like to schedule an appointment with our veterinary team "
            "to address these health findings and discuss the most effective treatment options for your pet?"
        )
        
        # Combine summary and question
        final_output = final_output = (
            f"Here are my findings: (Sypmtoms,treatment, clinical explaination)\n\n"
            f"{finding_s}"
            f"{appointment_question}"
             )
        print(final_output)
        def deduplicate_numbered_sections(text):
            # Split text into numbered blocks using regex
            blocks = re.split(r'(?=\d+\.)', text)
            seen = set()
            unique_blocks = []
            
            for block in blocks:
                cleaned_block = block.strip()
                if cleaned_block and cleaned_block not in seen:
                    unique_blocks.append(cleaned_block)
                    seen.add(cleaned_block)
            
            # Join back with line breaks to keep original format
            return '\n'.join(unique_blocks)
        final_output_dedup = deduplicate_numbered_sections(final_output)
        return final_output_dedup

# Initialize agents
classification_agent = ClassificationAgent(
    model=model_Class,
    tokenizer=tokenizer_class,
    device=device
)

retrieval_agent = RetrievalAgent(
    model=vetbert_model,
    tokenizer=vetbert_tokenizer,
    device=device,
    qdrant_client=client,
    collection_name=collection_name
)

communicationAgent = communicationAgent(
    model=model_textg,
    tokenizer=tokenizer_textg,
    device=device,
)

In [195]:
# Example usage with the agent system
def process_user_query(user_input: str):
    """Process user query through both agents"""
    print(f"Processing query: {user_input}\n")
    
    try:
        # Step 1: Use Classification Agent to identify the condition
        condition, confidence = classification_agent.identify_condition(user_input)
        # print(f"Classification Results:")
        # print(f"Identified Condition: {condition}")
        # print(f"Confidence Score: {confidence:.4f}\n")
        
        # Step 2: Use Retrieval Agent to find similar cases
        similar_cases = retrieval_agent.find_similar_cases(user_input, condition)
        
        # print(f"Similar Cases for {condition}:")
        # for case in similar_cases:
        #     print(f"Score: {case.score:.4f}")
        #     print(f"Case: {case.payload['text']}")
        #     print(f"Condition: {case.payload['condition']}\n")

        #step 3 explain the sitution in a simple words using communicationAgent
        # Carefully craft a conversational question
        explaination = communicationAgent.output_vet_assistant( similar_cases)
        print(f"explaination:\n{explaination}\n")
            
    except Exception as e:
        print(f"Error processing query: {str(e)}")

# Test the system
#user_input = "Something grows on one of my dog ears"
#process_user_query(user_input)

#Try another example
user_input2 = "My cat has been scratching a lot and has some red spots"
process_user_query(user_input2)

# Another example
# user_input3 = "My dog has been vomiting and has diarrhea"
# process_user_query(user_input3)

Processing query: My cat has been scratching a lot and has some red spots



INFO:__main__:Successfully generated prediction with confidence 0.8247
INFO:__main__:Identified condition: parasites with confidence: 0.8247
C:\Users\fhoss\AppData\Local\Temp\ipykernel_31184\3267297700.py:107: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = self.client.search(
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/vet_notes/points/search "HTTP/1.1 200 OK"
INFO:__main__:Found 3 similar cases for condition: parasites


Here are my findings: (Sypmtoms,treatment, clinical explaination)

1.flea dirt present on coat combing; ctenocephalides felis infestation confirmed.
ctenocephalides felis is a species of flies in the family Ctenocephalides.
2.sarcoptic mange; prescribe oral ivermectin.
sarcoptic mange is a cutaneous condition characterized by a rash, rash, and a rash that is characterized by a rash and rash that is characterized by a rash
3.ear mites in canal; clean and apply selamectin.
Selamectin is an antibiotic that can be used to treat ear mites in the canal.


Would you like to schedule an appointment with our veterinary team to address these health findings and discuss the most effective treatment options for your pet?
explaination:
Here are my findings: (Sypmtoms,treatment, clinical explaination)
1.flea dirt present on coat combing; ctenocephalides felis infestation confirmed.
ctenocephalides felis is a species of flies in the family Ctenocephalides.
2.sarcoptic mange; prescribe oral ivermectin